In [1]:
import json, random, shutil, os
from pathlib import Path

# --- Paths (edit these two to match your structure) ---
JSON_PATH   = Path("data/grounded_reports_20240819 (1).json")
IMAGES_DIR  = Path("data/PadChest_GR")  # or "data/PadChest_GR_resized_512_512"
OUT_ROOT    = Path("datasets/padchest_yolo")  # output dataset root

# Split ratios
SPLITS = {"train": 0.8, "val": 0.1, "test": 0.1}
random.seed(42)

# --- Load JSON ---
with open(JSON_PATH, "r", encoding="utf-8") as f:
    items = json.load(f)

# --- Collect labels (classes) from findings ---
label_set = set()
for it in items:
    for fnd in it.get("findings", []):
        for lab in fnd.get("labels", []):
            label_set.add(lab.strip())

# Make stable, deterministic ordering
names = sorted(label_set)
name_to_id = {n: i for i, n in enumerate(names)}

# --- Prepare output dirs ---
for split in ["train","val","test"]:
    (OUT_ROOT / "images" / split).mkdir(parents=True, exist_ok=True)
    (OUT_ROOT / "labels" / split).mkdir(parents=True, exist_ok=True)

# --- Helper: convert x1y1x2y2 (normalized) -> xc,yc,w,h (normalized) ---
def to_yolo(x1, y1, x2, y2):
    xc = (x1 + x2) / 2.0
    yc = (y1 + y2) / 2.0
    w  = (x2 - x1)
    h  = (y2 - y1)
    return xc, yc, w, h

# --- Build list of available images from JSON that actually exist ---
records = []
for it in items:
    img_name = it.get("ImageID")
    if not img_name:
        continue
    src_img = IMAGES_DIR / img_name
    if not src_img.is_file():
        # silently skip images that are not in the chosen images folder
        continue

    # Build all YOLO bboxes for this image
    yolo_lines = []
    for fnd in it.get("findings", []):
        boxes = fnd.get("boxes", []) or []
        labels = fnd.get("labels", []) or []

        # Decide the class label to use:
        # - If multiple labels are present, use the FIRST (customize if needed)
        # - If no labels, skip the finding
        cls_name = labels[0].strip() if labels else None
        if cls_name is None or cls_name not in name_to_id:
            continue
        cls_id = name_to_id[cls_name]

        for b in boxes:
            if len(b) != 4:
                continue
            x1, y1, x2, y2 = map(float, b)
            # Skip degenerate boxes
            if x2 <= x1 or y2 <= y1:
                continue
            xc, yc, w, h = to_yolo(x1, y1, x2, y2)
            # keep only boxes that are inside [0,1]
            if not (0 <= xc <= 1 and 0 <= yc <= 1 and 0 < w <= 1 and 0 < h <= 1):
                continue
            yolo_lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

    # keep images that have at least one box
    if yolo_lines:
        records.append((src_img, yolo_lines))

# --- Train/Val/Test split ---
random.shuffle(records)
n = len(records)
n_train = int(n * SPLITS["train"])
n_val   = int(n * SPLITS["val"])
splits = {
    "train": records[:n_train],
    "val":   records[n_train:n_train+n_val],
    "test":  records[n_train+n_val:],
}

# --- Write files ---
for split, recs in splits.items():
    for src_img, yolo_lines in recs:
        dst_img = OUT_ROOT / "images" / split / src_img.name
        dst_lab = OUT_ROOT / "labels" / split / (src_img.stem + ".txt")

        # copy image
        shutil.copy2(src_img, dst_img)
        # write label txt
        with open(dst_lab, "w", encoding="utf-8") as f:
            f.write("\n".join(yolo_lines))

# --- Create data.yaml ---
yaml_text = (
    f"train: {str((OUT_ROOT / 'images' / 'train').as_posix())}\n"
    f"val: {str((OUT_ROOT / 'images' / 'val').as_posix())}\n"
    f"test: {str((OUT_ROOT / 'images' / 'test').as_posix())}\n"
    f"nc: {len(names)}\n"
    "names:\n" + "\n".join([f"  {i}: {n}" for i, n in enumerate(names)])
)
with open(OUT_ROOT / "data.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_text)

print(f"Done. Images with labels: {n}. data.yaml saved to {OUT_ROOT/'data.yaml'}")


Done. Images with labels: 3008. data.yaml saved to datasets\padchest_yolo\data.yaml


In [2]:
# Check GPU
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [1]:
import torch, torchvision
print("torch:", torch.__version__, "| cuda:", torch.version.cuda)
print("torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


torch: 2.5.1+cu121 | cuda: 12.1
torchvision: 0.20.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [1]:
# Step 1: Set environment variable FIRST (before any imports)
import os
os.environ['WANDB_DISABLED'] = 'true'
os.environ['WANDB_MODE'] = 'disabled'

# Step 2: Now import everything else
import torch
from ultralytics import YOLO

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

def on_fit_epoch_end(trainer):
    """Callback to print training metrics at the end of each epoch"""
    try:
        m = trainer.metrics
        
        # Get validation metrics (these are available after validation)
        map50 = m.get('metrics/mAP50(B)', None) or getattr(m, 'map50', None)
        map50_95 = m.get('metrics/mAP50-95(B)', None) or getattr(m, 'map', None)
        
        # Get training loss from the label_loss_items
        # Format: [box_loss, cls_loss, dfl_loss]
        if hasattr(trainer, 'label_loss_items') and trainer.label_loss_items is not None:
            losses = trainer.label_loss_items()
            box_loss = losses[0] if len(losses) > 0 else float('nan')
            cls_loss = losses[1] if len(losses) > 1 else float('nan')
            dfl_loss = losses[2] if len(losses) > 2 else float('nan')
        else:
            box_loss = cls_loss = dfl_loss = float('nan')
        
        print(
            f"Epoch {trainer.epoch + 1}/{trainer.epochs} | "
            f"Train - box:{box_loss:.4f} cls:{cls_loss:.4f} dfl:{dfl_loss:.4f} | "
            f"Val - mAP50:{(map50 if map50 is not None else float('nan')):.4f} "
            f"mAP50-95:{(map50_95 if map50_95 is not None else float('nan')):.4f}"
        )
    except Exception as e:
        print(f"Callback error: {e}")

model = YOLO("yolov8m.pt")
model.add_callback("on_fit_epoch_end", on_fit_epoch_end)

results = model.train(
    data="datasets/padchest_yolo/data.yaml",
    epochs=40,            # fewer epochs
    patience=10,          # early stopping if no improvement for 10 epochs
    imgsz=640,
    batch=8,              # adjust for VRAM
    device=0,             # use first GPU
    workers=8,
    optimizer="AdamW",
    lr0=1e-3,
    cos_lr=True,
    amp=True,
    seed=0,
    deterministic=True,
    compile=False         # skip torch.compile to avoid dynamo issues
)

CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Ultralytics 8.3.204  Python-3.12.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=datasets/padchest_yolo/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train26, nbs=64, nms=False, opset=None, optimize

In [4]:
from ultralytics import YOLO

run_dir = "detect/train26"          # adjust to your folder
model   = YOLO(f"{run_dir}/weights/last.pt")   # use last.pt if best.pt not present

metrics = model.val()
print(metrics)


Ultralytics 8.3.204  Python-3.12.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Model summary (fused): 92 layers, 25,928,926 parameters, 0 gradients, 79.2 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1531.4223.0 MB/s, size: 6502.0 KB)
val: Scanning D:\Projects\projet_tuteuré\datasets\padchest_yolo\labels\val.cache... 300 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300  0.0sScanning D:\Projects\projet_tuteuré\datasets\padchest_yolo\labels\val.cache... 300 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300 126.5Kit/s 0.0s
val: D:\Projects\projet_tuteur\datasets\padchest_yolo\images\val\290116899728494592837689617318363239261_uhft0t.png: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 0.5it/s 37.1s2.5ss
                   all        300        787      0.496      0.164      0.166     0.0709
              NSG tube          8          8    

In [4]:
import os
from collections import Counter

# 👇 CHANGE THIS to your actual path to the "labels/train" folder
labels_dir = r"datasets/padchest_yolo/labels/train"

class_counts = Counter()

# Count class occurrences
for file in os.listdir(labels_dir):
    if file.endswith(".txt"):
        with open(os.path.join(labels_dir, file), "r") as f:
            for line in f:
                if line.strip():
                    class_id = line.strip().split()[0]  # first number = class id
                    class_counts[class_id] += 1

# Filter classes with >20 samples
filtered = {cid: count for cid, count in class_counts.items() if count >= 20}

print(f"Total classes: {len(class_counts)}")
print(f"Classes with >20 samples: {len(filtered)}\n")

for cid, count in sorted(filtered.items(), key=lambda x: -x[1]):
    print(f"Class {cid}: {count} samples")


Total classes: 142
Classes with >20 samples: 61

Class 35: 395 samples
Class 12: 373 samples
Class 125: 299 samples
Class 146: 261 samples
Class 7: 251 samples
Class 14: 239 samples
Class 151: 209 samples
Class 111: 198 samples
Class 10: 195 samples
Class 79: 192 samples
Class 44: 188 samples
Class 48: 163 samples
Class 8: 161 samples
Class 82: 133 samples
Class 58: 132 samples
Class 78: 124 samples
Class 34: 117 samples
Class 31: 111 samples
Class 70: 108 samples
Class 117: 108 samples
Class 77: 96 samples
Class 71: 91 samples
Class 101: 83 samples
Class 66: 81 samples
Class 69: 73 samples
Class 52: 72 samples
Class 107: 70 samples
Class 68: 65 samples
Class 19: 64 samples
Class 89: 63 samples
Class 130: 62 samples
Class 153: 54 samples
Class 100: 53 samples
Class 25: 48 samples
Class 141: 45 samples
Class 60: 43 samples
Class 53: 41 samples
Class 40: 37 samples
Class 1: 37 samples
Class 29: 34 samples
Class 106: 34 samples
Class 57: 33 samples
Class 63: 32 samples
Class 46: 30 sample

In [ ]:
import os, shutil, yaml
from pathlib import Path
from collections import Counter

# --------- CONFIG: original and new dataset roots ----------
old_root = Path(r"datasets/padchest_yolo")                 # has images/, labels/, data.yaml
new_root = Path(r"datasets/padchest_yolo_filtered")        # will be created
old_labels = old_root / "labels"
old_images = old_root / "images"
new_labels = new_root / "labels"
new_images = new_root / "images"
new_labels.mkdir(parents=True, exist_ok=True)
new_images.mkdir(parents=True, exist_ok=True)

# --------- Step 1: count class freq from TRAIN labels ----------
train_labels = old_labels / "train"
class_counts = Counter()
for file in train_labels.glob("*.txt"):
    for line in file.read_text().splitlines():
        if not line.strip(): 
            continue
        cls_id = line.split()[0]  # YOLO line: class xc yc w h (all normalized)  # docs: normalized xywh format
        class_counts[cls_id] += 1  # Ultralytics format details: docs.ultralytics.com/reference/engine/results/ etc.  # noqa

# keep classes with >=20 samples
valid_classes = sorted([cid for cid, c in class_counts.items() if c >= 20], key=int)
id_map = {old: str(i) for i, old in enumerate(valid_classes)}  # old -> new contiguous id

print(f"Total classes in train: {len(class_counts)}")
print(f"Kept classes (>=20): {len(valid_classes)}")
print("Remap (old->new):", id_map)

# --------- Step 2: filter & remap labels for all splits ----------
def filter_split(split: str):
    (new_labels / split).mkdir(parents=True, exist_ok=True)
    kept_files = 0
    for file in (old_labels / split).glob("*.txt"):
        out_lines = []
        for line in file.read_text().splitlines():
            parts = line.strip().split()
            if not parts: 
                continue
            cls_id = parts[0]
            if cls_id in id_map:
                parts[0] = id_map[cls_id]
                out_lines.append(" ".join(parts))
        if out_lines:  # only write labels if at least 1 object remains
            (new_labels / split / file.name).write_text("\n".join(out_lines))
            kept_files += 1
    print(f"{split}: wrote {kept_files} filtered label files")

for split in ["train", "val", "test"]:
    filter_split(split)

# --------- Step 3: copy only images that still have labels ----------
COMMON_EXTS = [".png", ".jpg", ".jpeg", ".bmp", ".webp", ".tif", ".tiff"]

def copy_images_for_split(split: str):
    (new_images / split).mkdir(parents=True, exist_ok=True)
    copied = 0
    for lbl in (new_labels / split).glob("*.txt"):
        stem = lbl.stem
        # find the matching image in the original split using common extensions
        src = None
        for ext in COMMON_EXTS:
            candidate = old_images / split / f"{stem}{ext}"
            if candidate.exists():
                src = candidate
                break
        if src is None:
            # try any file with that stem (slower fallback)
            matches = list((old_images / split).glob(stem + ".*"))
            src = matches[0] if matches else None
        if src:
            dst = new_images / split / src.name
            if not dst.exists():
                shutil.copy2(src, dst)
                copied += 1
    print(f"{split}: copied {copied} images")

for split in ["train", "val", "test"]:
    copy_images_for_split(split)

# --------- Step 4: build names and write data_filtered.yaml ----------
# read original names from old data.yaml
orig_yaml = old_root / "data.yaml"
with open(orig_yaml, "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

# original names may be list or dict
orig_names = {}
if isinstance(cfg.get("names"), list):
    orig_names = {i: n for i, n in enumerate(cfg["names"])}
elif isinstance(cfg.get("names"), dict):
    orig_names = {int(k): v for k, v in cfg["names"].items()}

# new contiguous names in new-id order
new_names = {int(new): orig_names.get(int(old), f"class_{old}") 
             for old, new in sorted(id_map.items(), key=lambda kv: int(kv[1]))}

data_filtered = {
    # make paths RELATIVE to this yaml's directory (Ultralytics resolves relative to YAML)
    # docs: data.yaml expects train/val/test, nc, names
    "train": "images/train",
    "val":   "images/val",
    "test":  "images/test",
    "nc":    len(new_names),
    "names": new_names,
}
with open(new_root / "data_filtered.yaml", "w", encoding="utf-8") as f:
    yaml.safe_dump(data_filtered, f, sort_keys=False)

print("\n✅ Wrote:", new_root / "data_filtered.yaml")


Total classes in train: 142
Kept classes (>=20): 61
Remap (old->new): {'1': '0', '7': '1', '8': '2', '10': '3', '11': '4', '12': '5', '14': '6', '19': '7', '23': '8', '25': '9', '26': '10', '27': '11', '29': '12', '31': '13', '34': '14', '35': '15', '40': '16', '44': '17', '46': '18', '48': '19', '51': '20', '52': '21', '53': '22', '57': '23', '58': '24', '60': '25', '63': '26', '64': '27', '66': '28', '68': '29', '69': '30', '70': '31', '71': '32', '75': '33', '76': '34', '77': '35', '78': '36', '79': '37', '82': '38', '89': '39', '96': '40', '99': '41', '100': '42', '101': '43', '106': '44', '107': '45', '111': '46', '117': '47', '119': '48', '124': '49', '125': '50', '127': '51', '130': '52', '133': '53', '134': '54', '141': '55', '144': '56', '145': '57', '146': '58', '151': '59', '153': '60'}
train: wrote 2316 filtered label files
val: wrote 285 filtered label files
test: wrote 282 filtered label files
train: copied 2316 images
val: copied 285 images
test: copied 282 images

✅ Wro

In [1]:
# Custom trainer that injects focal loss (gamma + alpha) into YOLOv8 detection loss
import os, torch
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
if hasattr(torch, "compiler"):
    torch.compiler.disable()

from ultralytics.cfg import DEFAULT_CFG  # <-- important fix
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics.utils.loss import v8DetectionLoss, FocalLoss

class V8DetLossWithAlpha(v8DetectionLoss):
    """
    Wrap YOLOv8 detection loss and replace/wrap the classification term with FocalLoss
    so we can set both gamma and alpha. Internal names can vary slightly by version.
    """
    def __init__(self, model, gamma=2.0, alpha=0.25):
        super().__init__(model)
        self._gamma = gamma
        self._alpha = alpha

        # Common path: BCE classification loss exposed as self.BCEcls
        if hasattr(self, "BCEcls") and self.BCEcls is not None:
            self.BCEcls = FocalLoss(self.BCEcls, gamma=self._gamma, alpha=self._alpha)
        # Some builds expose a focal wrapper as self.fl; set its attributes if present
        elif hasattr(self, "fl") and self.fl is not None:
            if hasattr(self.fl, "gamma"):
                self.fl.gamma = self._gamma
            if hasattr(self.fl, "alpha"):
                self.fl.alpha = self._alpha
        # Otherwise base loss proceeds unchanged (rare across recent versions)

class CustomTrainer(DetectionTrainer):
    def __init__(self, cfg=None, overrides=None, _callbacks=None, gamma=2.0, alpha=0.25):
        # Use DEFAULT_CFG when cfg is None to satisfy trainer init
        super().__init__(cfg=(cfg or DEFAULT_CFG), overrides=overrides, _callbacks=_callbacks)
        self._gamma = gamma
        self._alpha = alpha

    def build_loss(self, model):
        return V8DetLossWithAlpha(model, gamma=self._gamma, alpha=self._alpha)


In [2]:
from __main__ import CustomTrainer as _CT  # using the class defined in the current notebook

overrides = {
    "model": "yolov8m.pt",
    "data":  "datasets/padchest_yolo_filtered/data_filtered.yaml",
    "epochs": 100,
    "patience": 15,
    "imgsz": 640,
    "batch": -1,      # auto-batch
    "device": 0,
    "optimizer": "AdamW",
    "lr0": 1e-3,
    "cos_lr": True,
    "amp": True,

    # ---- loss weighting & regularization ----
    "box": 7.5,
    "cls": 1.0,
    "dfl": 1.5,
    "label_smoothing": 0.05,

    # ---- NO heavy augmentations for medical images ----
    "fliplr": 0.0,        # set to 0.5 only if left/right flips are clinically acceptable
    "scale": 0.0,
    "translate": 0.0,
    "auto_augment": None,
    "erasing": 0.0,

    # ---- stability ----
    "compile": False
}

trainer = _CT(overrides=overrides, gamma=2.0, alpha=0.25)  # focal loss params here
trainer.train()
print("Best weights:", trainer.best)


WARNING 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.204  Python-3.12.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=1.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=datasets/padchest_yolo_filtered/data_filtered.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.0, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train32, nbs=64, nms=Fal

In [3]:
from ultralytics import YOLO

# Load your trained model
model = YOLO(r"best.pt")

# Validate on the val set defined in your dataset yaml
metrics = model.val()
print(metrics)  # dict with precision, recall, mAPs, etc.


Ultralytics 8.3.204  Python-3.12.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Model summary (fused): 92 layers, 25,875,079 parameters, 0 gradients, 78.9 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1833.3366.5 MB/s, size: 7724.2 KB)
val: Scanning D:\Projects\projet_tuteuré\datasets\padchest_yolo_filtered\labels\val.cache... 285 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 285/285  0.0sScanning D:\Projects\projet_tuteuré\datasets\padchest_yolo_filtered\labels\val.cache... 285 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 285/285 54.6Kit/s 0.0s
val: D:\Projects\projet_tuteur\datasets\padchest_yolo_filtered\images\val\290116899728494592837689617318363239261_uhft0t.png: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 18/18 1.2it/s 14.9s0.8s
                   all        285        713      0.287      0.227      0.153     0.0717
              NSG tube 

In [4]:
import pandas as pd

# metrics.box maps: {class_id: [precision, recall, mAP50, mAP50-95]}
class_metrics = metrics.box.maps  # array of mAP per class
names = model.names               # id -> class name

data = []
for i, ap in enumerate(class_metrics):
    data.append({
        "class_id": i,
        "class_name": names[i],
        "mAP50-95": ap
    })

df = pd.DataFrame(data)
print(df.sort_values("mAP50-95", ascending=False))

# Optional: export to CSV
df.to_csv("per_class_metrics.csv", index=False)


    class_id                        class_name  mAP50-95
22        22               dual chamber device  0.510873
39        39                mammary prosthesis  0.471425
20        20      descendent aortic elongation  0.335001
15        15                      cardiomegaly  0.324832
30        30                     hiatal hernia  0.198613
..       ...                               ...       ...
26        26                            goiter  0.000000
40        40                             metal  0.000000
49        49             sclerotic bone lesion  0.000000
53        53  superior mediastinal enlargement  0.000000
54        54           supra aortic elongation  0.000000

[61 rows x 3 columns]


In [6]:
results = model.predict(
    source="datasets/padchest_yolo_filtered/images/val/1800719804166270189511533197998289596_f425d7.png",
    save=True,
    conf=0.25
)
results[0].show()  # display inline if running in a notebook



image 1/1 d:\Projects\projet_tuteur\datasets\padchest_yolo_filtered\images\val\1800719804166270189511533197998289596_f425d7.png: 640x640 2 costophrenic angle bluntings, 1 scoliosis, 27.8ms
Speed: 3.7ms preprocess, 27.8ms inference, 40.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\pc\runs\detect\predict9


In [10]:
# Inspect keys inside the summary
per_class_results = metrics.summary()
print(per_class_results[0])   # show the structure of the first class dictionary


{'Class': 'NSG tube', 'Images': np.int64(8), 'Instances': np.int64(8), 'Box-P': np.float64(0.12951), 'Box-R': np.float64(0.25), 'Box-F1': np.float64(0.17063), 'mAP50': np.float64(0.11791), 'mAP50-95': np.float64(0.03626)}
